In [32]:
!pip install elevenlabs python-dotenv -q

In [73]:
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs.play import play
import os
import pandas as pd
import IPython
import string
import re
import shutil
import subprocess


dot = load_dotenv()

In [34]:
# Prepare Medicine Data
raw = pd.read_csv("./medicine.csv")["med_name"]
raw

0                          (Bofalogn)Paracetamol Infusion
1                                   25%+Cevit+B6+B12+20cc
2                                           25%+Poly+20cc
3                                                   3-way
4                                   50%+Cevit+B6+B12+20cc
5                                            Abhayrch inj
6                                    Acetylcysteine 600mg
7                                         Aciclovir Cream
8                                           Aciclovir Tab
9                                            Acinet 625mg
10                                          Aclo100mg Tab
11                                          Actemra 400mg
12                                   Actilosa (prebiotic)
13                                        Acyclovir (inj)
14                                          Adenosine 6mg
15                                          Adernalin inj
16                                              Adult 50+
17            

In [61]:
# Clean Medicine Data
clean_data = []
raw_data = []
punctuation = ''.join([c for c in string.punctuation if c != '-'])
exclude = ('inj', 'tab', 'mg', 'ml', 'syp', 'mgtab', 'g', 'l', 'cc', 'dt', 'iv')
med_one_words = ['C', 'E', 'D', 'A', 'B']
for data in raw:
    word = ""
    for c in data:
        if c in string.ascii_letters + '-' + string.digits:
            word += c
        elif c in punctuation + string.whitespace:
            word += " "

    if word != "":
        words = word.strip().split(' ')

        _words = []
        for word in words:
            if word.lower() not in exclude and (len(word) > 1 or word in med_one_words):
                w_ = re.sub(r'\b\d+[a-zA-Z]+\b', '', word)
                if not w_.isdigit():
                    _words.append(w_)
        
        cleaned = " ".join(_words).strip()
        
        if cleaned != '':
            clean_data.append(cleaned)
            raw_data.append(data)



In [67]:
# Compare with raw data
pd.DataFrame(zip(clean_data, raw_data))

,0,1
0,Bofalogn Paracetamol Infusion,(Bofalogn)Paracetamol Infusion
1,Cevit B6 B12,25%+Cevit+B6+B12+20cc
2,Poly,25%+Poly+20cc
3,3-way,3-way
4,Cevit B6 B12,50%+Cevit+B6+B12+20cc
5,Abhayrch,Abhayrch inj
6,Acetylcysteine,Acetylcysteine 600mg
7,Aciclovir Cream,Aciclovir Cream
8,Aciclovir,Aciclovir Tab
9,Acinet,Acinet 625mg


In [68]:
# Final Clean
clean_data = list(set(clean_data))
clean_data.sort()
pd.DataFrame(clean_data)

,0
0,3-way
1,ASO
2,ASTYMIN -3 Drip
3,ATT
4,AVAM
5,Abhayrch
6,Acetylcysteine
7,Aciclovir
8,Aciclovir Cream
9,Acinet


In [62]:
# Agents
elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

voice_ids = {
    "Default": "JBFqnCBsd6RMkjVDRZzb",
    "AsianMan1": "K8elrI3roCHJugSjT3np",
    "Matilda": "XrExE9yKIg1WjnnlVkGX",
    "Serena": "pMsXgVXv3BLzUgSXRplE",
    "Daniel": "onwK4e9ZLuTAKqWW03F9",
    "Will": "bIHbv24MWmeRgasZH58o",
    "Laura": "FGY2WhTYpPnrIDTdsKH5",
    "Roger": "CwhRBWXzGAHq8TQ4Fs17",
    "Clara": "Qggl4b0xRMiqOwhPtVWT",
    "Arabella": "Z3R5wn05IrDiVCyEkUrK",
    # "Brittney": "pjcYQlDFKMbcOUp6F5GD",
    # "Ralf Eisent": "A9evEp8yGjv4c3WsIKuY",
    # "Trinity": "2qfp6zPuviqeCOZIE9RZ",
    # "Julia": "tOuLUAIdXShmWH7PEUrU",
    # "Liam": "TX3LPaxmHKxFdv7VOQHJ",
}

current_voice_id = "AsianMan1"
models = ["eleven_multilingual_v2", "eleven_flash_v2"]
current_model = "eleven_flash_v2"

def TTS(text, voice_id):
    audio_stream = elevenlabs.text_to_speech.convert(
        text=text,
        voice_id=voice_id,
        model_id=current_model,
        output_format="mp3_44100_128",
    )
    
    result = bytearray()
    
    for chunk in audio_stream:
        result.extend(chunk)
    
    return bytes(result)


In [51]:
# Voice Testing

audio = TTS('Celecoxid CELEC', voice_ids[current_voice_id])

with open('a.mp3', 'wb') as f:
    f.write(audio)

IPython.display.Audio('a.mp3')

In [63]:
# Voice Generation with one voice agent for all dataset

# voices_dir = os.path.join(os.getcwd(), 'voices', 'en')
# agent_dir = os.path.join(voices_dir, current_voice_id)


# for text in clean_data:
    
#     if not os.path.exists(agent_dir):
#         os.makedirs(agent_dir)
    
#     audio_path = os.path.join(agent_dir, f'{text}.mp3')
    
#     if not os.path.exists(audio_path):
#         audio = TTS(text, voice_ids[current_voice_id])
    
#         with open(audio_path, 'wb') as f:
#             f.write(audio)

In [64]:
# Voice Generation for one text for all agent

# text = "Paracetamol"
# voices_dir = os.path.join(os.getcwd(), 'voices', 'en')

# for voice_id in voice_ids.keys():
#     agent_dir = os.path.join(voices_dir, voice_id)

#     if not os.path.exists(agent_dir):
#         os.makedirs(agent_dir)
    
#     audio_path = os.path.join(agent_dir, f'{text}.mp3')
    
#     if not os.path.exists(audio_path):
#         audio = TTS(text, voice_ids[voice_id])
    
#         with open(audio_path, 'wb') as f:
#             f.write(audio)

In [105]:
# Create ASR dataset for one word

voices_dir = os.path.join(os.getcwd(), 'voices', 'en')
dataset_path = os.path.join(voices_dir, 'dataset')
data_path = os.path.join(dataset_path, 'audio')
metadata_path = os.path.join(dataset_path, 'metadata.csv')
meta_data_write_method = 'w'

text = "Paracetamol"

if os.path.exists(metadata_path) and meta_data_write_method != 'w':
    metadata_file = open(metadata_path,  meta_data_write_method)
else:
    metadata_file = open(metadata_path, 'w')
    metadata_file.write('file,text\n')

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

if not os.path.exists(data_path):
    os.makedirs(data_path)

for voice_id in voice_ids.keys():
    agent_dir = os.path.join(voices_dir, voice_id)

    if not os.path.exists(agent_dir):
        os.makedirs(agent_dir)
    
    audio_path = os.path.join(agent_dir, f'{text}.mp3')
    target_path = os.path.join(data_path, f"{text}.mp3")
    target_path_wav = os.path.join(data_path, f"{text}_{voice_id}.wav")
    
    if os.path.exists(audio_path) and not os.path.exists(target_path):
        shutil.copy(audio_path, target_path)
        try:
            subprocess.run(["ffmpeg", "-i", target_path, target_path_wav], 
                           stdout=subprocess.DEVNULL, 
                           stderr=subprocess.DEVNULL, 
                           check=True) # convert to wav
            os.remove(target_path) # remove mp3
            metadata_file.write(f"{text}_{voice_id}.wav,{text}\n") # csv
        except subprocess.CalledProcessError as e:
            print("Error:", e.stderr)


metadata_file.close()
pd.read_csv(metadata_path)

,file,text
0,Paracetamol_Default.wav,Paracetamol
1,Paracetamol_AsianMan1.wav,Paracetamol
2,Paracetamol_Matilda.wav,Paracetamol
3,Paracetamol_Serena.wav,Paracetamol
4,Paracetamol_Daniel.wav,Paracetamol
5,Paracetamol_Will.wav,Paracetamol
6,Paracetamol_Laura.wav,Paracetamol
7,Paracetamol_Roger.wav,Paracetamol
8,Paracetamol_Clara.wav,Paracetamol
9,Paracetamol_Arabella.wav,Paracetamol
